# How to Get Started with the Model
We floows the steps on the HuggingFace webpage for the [Florence-2](https://huggingface.co/microsoft/Florence-2-large-ft).

First import libraries.

In [ ]:
# instal libraries if needed with pip -----------------------------------------#
%pip install requests
%pip install pillow
%pip install transformers
%pip install wheel
%pip install flash_attn 
%pip install timm 
%pip install einops

# load libraries --------------------------------------------------------------#
import requests
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM
import torch
print(torch.rand(3))

# Check whether cuda is available for faster processing -----------------------#
torch.cuda.is_available()

In [6]:

model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)

prompt = "<OD>"

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=prompt, images=image, return_tensors="pt")

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    do_sample=False,
    num_beams=3
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))

print(parsed_answer)


{'<OD>': {'bboxes': [[34.23999786376953, 160.0800018310547, 597.4400024414062, 371.7599792480469], [454.7200012207031, 97.19999694824219, 579.5199584960938, 261.3599853515625], [452.79998779296875, 276.7200012207031, 553.9199829101562, 370.79998779296875], [94.4000015258789, 280.55999755859375, 196.1599884033203, 371.2799987792969]], 'labels': ['car', 'door', 'wheel', 'wheel']}}
